In [0]:
from tensorflow import keras
import tensorflow as tf

from keras import layers
from keras import models
from keras import optimizers

import matplotlib.pylab as plt
import numpy as np
  
from google.colab import drive
import os
import h5py

Using TensorFlow backend.


In [0]:
if os.path.exists('/content/gdrive/')==False:
  drive.mount('/content/gdrive')
  print('Google drive is mounted\n')
else:
  print('Google drive is already mounted\n')

Google drive is already mounted



In [0]:
from keras.applications import densenet
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [0]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1/255)

In [0]:
train_path = '/content/gdrive/My Drive/food_an'
val_path = '/content/gdrive/My Drive/food_test/val_set'

train_dir = str(train_path)
val_dir = str(val_path)

In [0]:
image_size = 224
train_batchsize = 100
val_batchsize = 10

train_generator = train_datagen.flow_from_directory(train_dir,
                                                  target_size=(image_size,image_size),
                                                  batch_size=train_batchsize,
                                                  class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(val_dir,
                                                    target_size=(image_size,image_size),
                                                    batch_size=val_batchsize,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 83952 images belonging to 93 classes.
Found 9367 images belonging to 93 classes.


In [0]:
classes = 93
# dense = densenet.DenseNet201(weights='imagenet',input_shape=(image_size,image_size,3))

In [0]:
'''
for layer in dense.layers[:-2]:
  layer.trainable = False
  
for layer in dense.layers:
  print(layer, layer.trainable)
'''
# dense.trainable = False

'\nfor layer in dense.layers[:-2]:\n  layer.trainable = False\n  \nfor layer in dense.layers:\n  print(layer, layer.trainable)\n'

In [0]:
model = models.Sequential()

model_dense = keras.models.load_model('/content/gdrive/My Drive/save_file/densenet2.h5')

model_dense.summary()

model = model_dense

W0618 08:12:34.213572 139922459314048 nn_ops.py:4224] Large dropout rate: 0.7 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Model)          (None, 7, 7, 1920)        18321984  
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 7, 1920)        7680      
_________________________________________________________________
average_pooling2d_1 (Average (None, 4, 4, 1920)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 30720)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              31458304  
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)             

In [0]:
#model.add(layers.BatchNormalization())
#model.add(layers.AveragePooling2D(padding='same'))
'''
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.7))
model.add(layers.Dense(500, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.7))
model.add(layers.Dense(classes, activation='softmax'))

model.summary()
'''

In [0]:
import tensorflow.keras.backend as K
sess = K.get_session()
init = tf.global_variables_initializer()
sess.run(init)

In [0]:



#checkpoint_path = '/content/gdrive/My Drive/weights2.hdf5'
#model.load_weights(checkpoint_path)

ValueError: ignored

In [0]:
class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []
    
  def on_batch_end(self, batch, logs=None):
    self.batch_losses.append(logs['loss'])
    self.batch_acc.append(logs['acc'])
    
batch_stats = CollectBatchStats()
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/content/gdrive/My Drive/weights3.hdf5', verbose=1, save_freq = 'epoch')

In [0]:
board = tf.keras.callbacks.TensorBoard(log_dir='/content/gdrive/My Drive/tensorboard/logs',
                            histogram_freq=0,
                            batch_size=32, 
                            write_graph=True, 
                            write_grads=False, 
                            write_images=False, 
                            embeddings_freq=0, 
                            embeddings_layer_names=None, 
                            embeddings_metadata=None, 
                            embeddings_data=None, 
                            update_freq='epoch')

In [0]:

history = model.fit_generator(train_generator,
                              steps_per_epoch=train_generator.samples/(train_generator.batch_size),
                              epochs=20,
                              validation_data=validation_generator,
                              validation_steps=validation_generator.samples/validation_generator.batch_size,
                              verbose=1,
                              callbacks=[batch_stats, checkpointer, board])

Epoch 1/20


ValueError: ignored

In [0]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(batch_stats.batch_losses)

plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats.batch_acc)

In [0]:

model.save('/content/gdrive/My Drive/save_file/densenet2.h5')

In [0]:
model2 = keras.models.load_model('/content/gdrive/My Drive/save_file/densenet2.h5')
model2.summary()